In [165]:
import ccxt
import pandas as pd
import plotly.subplots as sp
import plotly.graph_objs as go
import streamlit as st
from streamlit_option_menu import option_menu
import ast

In [20]:
# Initialize the CCXT exchange object (use your desired exchange)
exchange = ccxt.binance()

SYMBOL = 'ADA/USDT'
TIMEFRAME = '3d'

START_DATE = '2016-01-01'

SMA1 = 10  # Customize the window for the first SMA
SMA2 = 16

In [161]:
def return_trending_coin_info():

    trending_coin_data = pd.DataFrame()
    for coin_data in cg.get_search_trending()['coins']:

        melted_trending_coin_df = pd.DataFrame(coin_data['item']).reset_index()
        melted_trending_coin_df['price_change_pct_btc_24h'] = melted_trending_coin_df[melted_trending_coin_df['index']=='price_change_percentage_24h']['data'].values[0]['btc']
        melted_trending_coin_df['price_change_pct_usd_24h'] = melted_trending_coin_df[melted_trending_coin_df['index']=='price_change_percentage_24h']['data'].values[0]['usd']

        melted_trending_coin_df = melted_trending_coin_df[~(melted_trending_coin_df['index'].isin(['sparkline', 'price_change_percentage_24h']))]

        trending_coin_info = melted_trending_coin_df.pivot_table(index=['id', 'symbol', 
                                                                        'market_cap_rank', 'score', 
                                                                        'price_change_pct_btc_24h', 'price_change_pct_usd_24h'], columns='index', values='data', aggfunc='first').reset_index()
        trending_coin_info['score'] = trending_coin_info['score'] + 1

        trending_coin_data = trending_coin_data.append(trending_coin_info)

    return trending_coin_data
    

In [162]:
trending_coin_data = return_trending_coin_info()

In [163]:
trending_coin_data

index,id,symbol,market_cap_rank,score,price_change_pct_btc_24h,price_change_pct_usd_24h,content,market_cap,market_cap_btc,price,price_btc,total_volume,total_volume_btc
0,gala,GALA,57,1,13.025782,17.261396,"{'title': 'What is GALA?', 'description': 'Gal...","$2,565,084,011",37172.2669930359,$0.07107,0.00000102927388844436,"$422,338,443",6116.71322243317
0,book-of-meme,BOME,140,2,165.850904,175.813603,NaN,"$754,690,275",10941.7146909847,$0.01410,0.000000204143078602714,"$1,304,988,745",18900.1073338318
0,tenset,10SET,327,3,7.527649,11.607527,NaN,"$212,368,236",3077.39156935855,$1.27,0.0000183640642042646,"$1,675,691",24.2580302014462
0,ponke,PONKE,752,4,-17.259547,-14.427374,NaN,"$51,132,512",740.962678075609,$0.1023,0.00000148100159880156,"$12,499,923",181.035962562815
0,mantra-dao,OM,245,5,15.711164,19.671912,NaN,"$342,531,617",4963.28159866502,$0.4265,0.00000617658439055773,"$50,711,321",734.450324825846
0,orion-protocol,ORN,569,6,11.446732,15.623172,NaN,"$84,212,975",1220.2456353422,$2.44,0.0000353132281723146,"$26,149,104",378.716582470989
0,seedify-fund,SFUND,249,7,7.493364,11.172819,NaN,"$333,033,927",4826.00406512286,$5.26,0.0000762390693473905,"$14,602,111",211.481873194358
0,dexcheck,DCK,862,8,-4.294738,-1.018784,NaN,"$40,202,349",582.865113723505,$0.1222,0.00000177053914348997,"$1,966,515",28.4809633906384
0,edu-coin,EDU,273,9,4.570978,8.150402,NaN,"$284,569,531",4123.87841993223,$1.09,0.0000158528465034124,"$19,733,349",285.797424815017
0,jupiter-exchange-solana,JUP,66,10,58.325989,64.259217,NaN,"$2,000,815,676",28991.8101606091,$1.49,0.000021635541101334,"$1,135,400,550",16443.967306819


In [21]:
def extract_data():

    crypto_df = pd.DataFrame()

    # Fetch historical OHLCV data in chunks and append to crypto_df
    # Calculate the number of milliseconds in the timeframe
    if TIMEFRAME == '4h':
        timeframe_ms = 4 * 60 * 60 * 1000
    elif TIMEFRAME == '12h':
        timeframe_ms = 12 * 60 * 60 * 1000
    elif TIMEFRAME == '1d':
        timeframe_ms = 24 * 60 * 60 * 1000
    elif TIMEFRAME == '3d':
        timeframe_ms = 3 * 24 * 60 * 60 * 1000
    elif TIMEFRAME == '1w':
        timeframe_ms = 7 * 24 * 60 * 60 * 1000
    elif TIMEFRAME == '1M':
        timeframe_ms = 30 * 24 * 60 * 60 * 1000

    # Initialize an empty DataFrame to store the combined data
    crypto_df = pd.DataFrame()

    start_date = START_DATE
    # Fetch historical OHLCV data in chunks and append to crypto_df
    while True:
        # Fetch historical OHLCV data with a limit based on the timeframe
        ohlcv = exchange.fetch_ohlcv(SYMBOL, TIMEFRAME, since=int(pd.Timestamp(start_date).timestamp() * 1000), limit=1000)

        # Convert OHLCV data to a DataFrame
        chunk_df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

        # Convert timestamps to datetime format
        chunk_df['timestamp'] = pd.to_datetime(chunk_df['timestamp'], unit='ms')

        # Append the chunk to crypto_df
        crypto_df = pd.concat([crypto_df, chunk_df], ignore_index=True)

        # Update the start_date for the next chunk
        start_date = crypto_df['timestamp'].iloc[-1] + pd.DateOffset(milliseconds=timeframe_ms)

        # If the data returned is less than the limit, it means we've fetched all available data
        if len(ohlcv) < 1000:
            break


    # Convert timestamps to datetime format
    crypto_df['timestamp'] = pd.to_datetime(crypto_df['timestamp'], unit='ms')


    # Calculate Simple Moving Averages (SMA)
    # Customize the window for the second SMA
    crypto_df['sma1'] = crypto_df['close'].rolling(window=SMA1).mean()
    crypto_df['sma2'] = crypto_df['close'].rolling(window=SMA2).mean()

    return crypto_df

    

In [22]:
def return_buy_and_sell_signals(crypto_df):
    # Initialize the 'Buy and Hold' and 'Sell and Observe' columns
    crypto_df['Buy Flag'] = 0

    # Flag the points where SMA1 crosses above SMA2 as 'Buy and Hold' (1)
    crypto_df.loc[crypto_df['sma1'] > crypto_df['sma2'], 'Buy Flag'] = 1

    crypto_df.sort_values('timestamp', inplace=True)

    crypto_df['Buy Flag Lagged'] = crypto_df['Buy Flag'].shift(1)

    buy_signals = crypto_df[(crypto_df['Buy Flag']==1)&(crypto_df['Buy Flag Lagged']==0)]
    sell_signals = crypto_df[(crypto_df['Buy Flag']==0)&(crypto_df['Buy Flag Lagged']==1)]

    return buy_signals, sell_signals
    

In [29]:
def plot_price_chart(crypto_df):

    #Extracting Buy and sell signals based on moving averages
    buy_signals_df, sell_signals_df = return_buy_and_sell_signals(crypto_df)


    # Create a subplot grid
    fig = sp.make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[0.7, 0.3])

    # Add the candlestick chart to the upper subplot with name 'Price'
    candlestick = go.Candlestick(x=crypto_df['timestamp'],
                                open=crypto_df['open'],
                                high=crypto_df['high'],
                                low=crypto_df['low'],
                                close=crypto_df['close'],
                                name='Price')
    fig.append_trace(candlestick, row=1, col=1)

    # Add SMAs to the candlestick chart with custom line colors
    sma1_trace = go.Scatter(x=crypto_df['timestamp'], y=crypto_df['sma1'], mode='lines', name=f'SMA-{SMA1}', line=dict(color='yellow'))
    sma2_trace = go.Scatter(x=crypto_df['timestamp'], y=crypto_df['sma2'], mode='lines', name=f'SMA-{SMA2}', line=dict(color='pink'))
    fig.add_trace(sma1_trace, row=1, col=1)
    fig.add_trace(sma2_trace, row=1, col=1)

    # Define the buy and sell signals as datetime values
    buy_signals = buy_signals_df['timestamp']
    sell_signals = sell_signals_df['timestamp']

    # Create vertical lines at buy and sell signals
    for buy_signal in buy_signals:
        fig.add_shape(go.layout.Shape(
            type="line",
            x0=buy_signal,
            x1=buy_signal,
            y0=crypto_df['low'].min(),
            y1=crypto_df['high'].max(),
            line=dict(color="green", dash="dot")
        ))

    for sell_signal in sell_signals:
        fig.add_shape(go.layout.Shape(
            type="line",
            x0=sell_signal,
            x1=sell_signal,
            y0=crypto_df['low'].min(),
            y1=crypto_df['high'].max(),
            line=dict(color="red", dash="dot")
        ))


    # Set the layout for the cryptocurrency chart
    fig.update_layout(title=f'{SYMBOL} Candlestick Chart ({TIMEFRAME})',
                    xaxis_rangeslider_visible=True)

    # Create a volume chart as a subplot in the lower subplot with name 'Volume'
    volume = go.Bar(x=crypto_df['timestamp'], y=crypto_df['volume'], marker_color='blue', name='Volume')
    fig.append_trace(volume, row=2, col=1)

    # Update the subplot height
    fig.update_layout(yaxis2=dict(domain=[0.1, 0.3]))

    # Add legends
    fig.update_layout(legend=dict(orientation="h", x=0, y=1.05), height=650, width=1000)

    # Display the chart
    fig.show()

    return buy_signals_df, sell_signals_df

In [30]:
crypto_df = extract_data()

buy_signals_df, sell_signals_df = plot_price_chart(crypto_df)

In [7]:
def return_agg_profit_pct(buy_signals_df, sell_signals_df):

    buy_signals_df_req = buy_signals_df[['timestamp', 'close']].copy()
    buy_signals_df_req['Action'] = 'BUY'
    sell_signals_df_req = sell_signals_df[['timestamp', 'close']].copy()
    sell_signals_df_req['Action'] = 'SELL'

    signal_df = pd.concat([buy_signals_df_req, sell_signals_df_req]).sort_values('timestamp')

    signal_df['Sell Date'] = signal_df['timestamp'].shift(-1)
    signal_df['Sell Price'] = signal_df['close'].shift(-1)


    signal_df_req = signal_df[signal_df['Action']=='BUY']
    signal_df_req.rename(columns={'close':'Buy Price'}, inplace=True)


    signal_df_req['Profit'] = signal_df_req['Sell Price'] - signal_df_req['Buy Price']
    signal_df_req['Profit_pct'] = (signal_df_req['Sell Price'] - signal_df_req['Buy Price'])/signal_df_req['Buy Price']

    print(f"Overall Profit Percentage:{signal_df_req['Profit_pct'].sum()*100:.2f}%")

    return signal_df_req

In [8]:
signal_df_req = return_agg_profit_pct(buy_signals_df, sell_signals_df)

Overall Profit Percentage:524.82%


/Users/triloksachin/Desktop/Work/Softwares/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/rv/kj5c_6812y1876mdxtkgw9740000gn/T/ipykernel_40758/4257679191.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/rv/kj5c_6812y1876mdxtkgw9740000gn/T/ipykernel_40758/4257679191.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

In [67]:
signal_df_req

,timestamp,Buy Price,Action,Sell Date,Sell Price,Profit,Profit_pct
19,2017-10-13,5709.99,BUY,2018-01-14,10900.00,5190.01,0.908935
68,2018-03-09,9533.57,BUY,2018-03-24,8134.23,-1399.34,-0.146780
85,2018-04-29,9071.48,BUY,2018-05-26,7099.00,-1972.48,-0.217438
113,2018-07-22,8397.24,BUY,2018-08-21,6525.01,-1872.23,-0.222958
131,2018-09-14,6505.00,BUY,2018-09-29,6611.61,106.61,0.016389
142,2018-10-17,6528.88,BUY,2018-10-29,6371.93,-156.95,-0.024039
170,2019-01-09,3601.31,BUY,2019-01-27,3411.04,-190.27,-0.052834
185,2019-02-23,3827.92,BUY,2019-07-29,10080.53,6252.61,1.633422
245,2019-08-22,10134.35,BUY,2019-09-06,10381.18,246.83,0.024356
270,2019-11-05,9216.20,BUY,2019-11-26,7419.49,-1796.71,-0.194951
